# Data mart for segmentation

In [85]:
df_clean = df_paid_10

In [86]:
df = df_clean
df = df.reset_index(drop=True)

In [87]:
df.head()

,Order_ID,Phone_new,Source,PaymentType,Region,FullSum,Quant,RowDiscount,RowSum,TN,TK
0,1303000526_TT,55575456-52485253545172,Онлайн-Резерв.,при получении,Центральный ф.о.,3498.0,1.0,0.0,2599.0,Мелкая бытовая техника,Приготовление пищи
1,1303000526_TT,55575456-52485253545172,Онлайн-Резерв.,при получении,Центральный ф.о.,3498.0,1.0,0.0,649.0,Аксессуары,Электропитание
2,1303000530_TT,55575050-50535349495672,Онлайн-Резерв.,при получении,Центральный ф.о.,54497.0,1.0,0.0,19999.0,Цифровая Техника,Мобильные средства связи
3,1303000530_TT,55575050-50535349495672,Онлайн-Резерв.,при получении,Центральный ф.о.,54497.0,1.0,0.0,19999.0,Цифровая Техника,Мобильные средства связи
4,1303000530_TT,55575050-50535349495672,Онлайн-Резерв.,при получении,Центральный ф.о.,54497.0,1.0,0.0,14499.0,Цифровая Техника,Мобильные средства связи


In [88]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22608 entries, 0 to 22607
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Order_ID     22608 non-null  object 
 1   Phone_new    22608 non-null  object 
 2   Source       22608 non-null  object 
 3   PaymentType  22608 non-null  object 
 4   Region       22608 non-null  object 
 5   FullSum      22608 non-null  float64
 6   Quant        22608 non-null  float64
 7   RowDiscount  22608 non-null  float64
 8   RowSum       22608 non-null  float64
 9   TN           22608 non-null  object 
 10  TK           22608 non-null  object 
dtypes: float64(4), object(7)
memory usage: 1.9+ MB


In [89]:
#column groupings
group_Source = df.groupby(['Phone_new', 'Source']).sum()
group_PaymentType = df.groupby(['Phone_new', 'PaymentType']).sum()
group_Region = df.groupby(['Phone_new', 'Region']).sum()
group_TN = df.groupby(['Phone_new', 'TN']).sum()

In [90]:
#groupings by mean and quantity
group_Phone_mean = df.groupby('Phone_new')[df.select_dtypes(include='number').columns].mean()
group_Phone_count = df.groupby(['Phone_new']).count()

In [91]:
#create a dataframe to record customer information
data_clients = pd.DataFrame()
data_clients['id'] = df.drop_duplicates(subset = 'Phone_new')['Phone_new']
data_clients = data_clients.reset_index(drop=True)

In [92]:
data_clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8772 entries, 0 to 8771
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      8772 non-null   object
dtypes: object(1)
memory usage: 68.7+ KB


*Add columns to separate categorical attributes.*

In [93]:
for i in df.TN.unique():
    data_clients[i] = i
    data_clients = data_clients.merge(group_TN['Quant'], left_on = ['id', i], right_on = ['Phone_new', 'TN'], how = 'left')
    data_clients = data_clients.drop(columns = [i])
    data_clients = data_clients.rename(columns = {'Quant': i})
    data_clients = data_clients.fillna(0)

In [94]:
for i in df.Source.unique():
    data_clients[i] = i
    data_clients = data_clients.merge(group_Source['Quant'], left_on = ['id', i], right_on = ['Phone_new', 'Source'], how = 'left')
    data_clients = data_clients.drop(columns = [i])
    data_clients = data_clients.rename(columns = {'Quant': i})
    data_clients = data_clients.fillna(0)

In [95]:
for i in df.PaymentType.unique():
    data_clients[i] = i
    data_clients = data_clients.merge(group_PaymentType['Quant'], left_on = ['id', i], right_on = ['Phone_new', 'PaymentType'], how = 'left')
    data_clients = data_clients.drop(columns = [i])
    data_clients = data_clients.rename(columns = {'Quant': i})
    data_clients = data_clients.fillna(0)

In [96]:
for i in df.Region.unique():
    data_clients[i] = i
    data_clients = data_clients.merge(group_Region['Quant'], left_on = ['id', i], right_on = ['Phone_new', 'Region'], how = 'left')
    data_clients = data_clients.drop(columns = [i])
    data_clients = data_clients.rename(columns = {'Quant': i})
    data_clients = data_clients.fillna(0)

In [97]:
#add main columns with sums and number of items
data_clients = data_clients.merge(df.groupby(['Phone_new']).sum(), left_on = ['id'], right_on = ['Phone_new'], how = 'left')

In [98]:
data_clients.head()

,id,Мелкая бытовая техника,Аксессуары,Цифровая Техника,Климат,Сделай сам,Детские товары,ТВ-Аудио,Крупная бытовая техника,Товары для дома,Спорт и активный отдых,Мебель,"Игры, софт и развлечения",Автотовары,Онлайн-Резерв.,Колл-центр,Сайт,Терминал,при получении,предоплата онлайн,Центральный ф.о.,Северо-Западный ф.о.,Сибирский ф.о.,Южный ф.о.,Уральский ф.о.,Приволжский ф.о.,Дальневосточный ф.о.,Order_ID,Source,PaymentType,Region,FullSum,Quant,RowDiscount,RowSum,TN,TK
0,55575456-52485253545172,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1303000526_TT1303000526_TT,Онлайн-Резерв.Онлайн-Резерв.,при получениипри получении,Центральный ф.о.Центральный ф.о.,6996.0,2.0,0.0,3248.0,Мелкая бытовая техникаАксессуары,Приготовление пищиЭлектропитание
1,55575050-50535349495672,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1303000530_TT1303000530_TT1303000530_TT,Онлайн-Резерв.Онлайн-Резерв.Онлайн-Резерв.,при получениипри получениипри получении,Центральный ф.о.Центральный ф.о.Центральный ф.о.,163491.0,3.0,0.0,54497.0,Цифровая ТехникаЦифровая ТехникаЦифровая Техника,Мобильные средства связиМобильные средства свя...
2,55574954-49565455575577,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1303000533_TT1303000533_TT,Онлайн-Резерв.Онлайн-Резерв.,при получениипри получении,Северо-Западный ф.о.Северо-Западный ф.о.,3296.0,2.0,140.0,1458.0,Цифровая ТехникаМелкая бытовая техника,Портативная электроникаПриготовление пищи
3,55574954-50515050575778,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,4.0,0.0,0.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,1303000539_TT1303000539_TT5539709_BX5539709_BX,Онлайн-Резерв.Онлайн-Резерв.Онлайн-Резерв.Онла...,при получениипри получениипри получениипри пол...,Северо-Западный ф.о.Северо-Западный ф.о.Северо...,9824.0,4.0,0.0,4722.0,АксессуарыАксессуарыМебельМебель,Аксессуары для комп./ноутбуков/принтеровАксесс...
4,55575050-50485353485575,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1303000540_TT1303000540_TT,Онлайн-Резерв.Онлайн-Резерв.,при получениипри получении,Центральный ф.о.Центральный ф.о.,1016.0,2.0,0.0,508.0,АксессуарыАксессуары,ЭлектропитаниеЭлектропитание


In [99]:
data_clients = data_clients.drop(columns=['FullSum', 'RowSum', 'Region', 'Order_ID', 'Source', 'PaymentType', 'TN',	'TK'])
data_clients.rename(columns = {'Quant' : 'КОЛ-ВО ВЫКУПЛ.ТОВ.'}, inplace = True) 
data_clients.rename(columns = {'RowDiscount' : 'СУММА СКИДКИ'}, inplace = True) 

In [100]:
# add a column with the number of cheques
data_clients = data_clients.merge(df.groupby(['Phone_new', 'Order_ID']).sum()['RowSum'].groupby('Phone_new').count(), left_on = ['id'], right_on = ['Phone_new'], how = 'left')

In [101]:
data_clients = data_clients.rename(columns={'RowSum':'КОЛ-ВО ЧЕКОВ'})

In [102]:
# average number of items in a cheque
data_clients['СРЕД.КОЛ.ТОВ.вЧеке'] = data_clients['КОЛ-ВО ВЫКУПЛ.ТОВ.'] / data_clients['КОЛ-ВО ЧЕКОВ']

In [103]:
# add a column with the average cheque
data_clients = data_clients.merge(df.groupby(['Phone_new', 'Order_ID']).sum()['RowSum'].groupby('Phone_new').mean(), left_on = ['id'], right_on = ['Phone_new'], how = 'left')
data_clients = data_clients.rename(columns={'RowSum':'СРЕДНИЙ ЧЕК'})

In [104]:
#refill TN columns with fractions instead of quantities
data_clients['Аксессуары'] = data_clients['Аксессуары'] / data_clients['КОЛ-ВО ВЫКУПЛ.ТОВ.']
data_clients['Крупная бытовая техника'] = data_clients['Крупная бытовая техника'] / data_clients['КОЛ-ВО ВЫКУПЛ.ТОВ.']
data_clients['Цифровая Техника'] = data_clients['Цифровая Техника'] / data_clients['КОЛ-ВО ВЫКУПЛ.ТОВ.']
data_clients['Мелкая бытовая техника'] = data_clients['Мелкая бытовая техника'] / data_clients['КОЛ-ВО ВЫКУПЛ.ТОВ.']
data_clients['ТВ-Аудио'] = data_clients['ТВ-Аудио'] / data_clients['КОЛ-ВО ВЫКУПЛ.ТОВ.']
data_clients['Мебель'] = data_clients['Мебель'] / data_clients['КОЛ-ВО ВЫКУПЛ.ТОВ.']
data_clients['Детские товары'] = data_clients['Детские товары'] / data_clients['КОЛ-ВО ВЫКУПЛ.ТОВ.']
data_clients['Спорт и активный отдых'] = data_clients['Спорт и активный отдых'] / data_clients['КОЛ-ВО ВЫКУПЛ.ТОВ.']
data_clients['Сделай сам'] = data_clients['Сделай сам'] / data_clients['КОЛ-ВО ВЫКУПЛ.ТОВ.']
data_clients['Климат'] = data_clients['Климат'] / data_clients['КОЛ-ВО ВЫКУПЛ.ТОВ.']
data_clients['Игры, софт и развлечения'] = data_clients['Игры, софт и развлечения'] / data_clients['КОЛ-ВО ВЫКУПЛ.ТОВ.']
data_clients['Товары для дома'] = data_clients['Товары для дома'] / data_clients['КОЛ-ВО ВЫКУПЛ.ТОВ.']
data_clients['Автотовары'] = data_clients['Автотовары'] / data_clients['КОЛ-ВО ВЫКУПЛ.ТОВ.']

In [105]:
#customer showcase
data_clients.head()

,id,Мелкая бытовая техника,Аксессуары,Цифровая Техника,Климат,Сделай сам,Детские товары,ТВ-Аудио,Крупная бытовая техника,Товары для дома,Спорт и активный отдых,Мебель,"Игры, софт и развлечения",Автотовары,Онлайн-Резерв.,Колл-центр,Сайт,Терминал,при получении,предоплата онлайн,Центральный ф.о.,Северо-Западный ф.о.,Сибирский ф.о.,Южный ф.о.,Уральский ф.о.,Приволжский ф.о.,Дальневосточный ф.о.,КОЛ-ВО ВЫКУПЛ.ТОВ.,СУММА СКИДКИ,КОЛ-ВО ЧЕКОВ,СРЕД.КОЛ.ТОВ.вЧеке,СРЕДНИЙ ЧЕК
0,55575456-52485253545172,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1,2.0,3248.0
1,55575050-50535349495672,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,1,3.0,54497.0
2,55574954-49565455575577,0.5,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,140.0,1,2.0,1458.0
3,55574954-50515050575778,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,4.0,0.0,0.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,2,2.0,2361.0
4,55575050-50485353485575,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1,2.0,508.0


In [106]:
df1 = data_clients.copy()

In [107]:
df = df1

In [108]:
df.set_index('id', inplace = True)

In [109]:
data_clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8772 entries, 0 to 8771
Data columns (total 32 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        8772 non-null   object 
 1   Мелкая бытовая техника    8772 non-null   float64
 2   Аксессуары                8772 non-null   float64
 3   Цифровая Техника          8772 non-null   float64
 4   Климат                    8772 non-null   float64
 5   Сделай сам                8772 non-null   float64
 6   Детские товары            8772 non-null   float64
 7   ТВ-Аудио                  8772 non-null   float64
 8   Крупная бытовая техника   8772 non-null   float64
 9   Товары для дома           8772 non-null   float64
 10  Спорт и активный отдых    8772 non-null   float64
 11  Мебель                    8772 non-null   float64
 12  Игры, софт и развлечения  8772 non-null   float64
 13  Автотовары                8772 non-null   float64
 14  Онлайн-Р